In [20]:
from openai import OpenAI
from dill.source import getsource
import os
# make sure OPENAI_API_KEY environment variable is set
import site
site.addsitedir('../')
from importlib import reload
from gsm2py import model
reload(model)
with open('../gsm2py/data/enc_directive','r') as f:
    enc_dir=f.read()
with open('../gsm2py/data/dec_directive','r') as f:
    dec_dir=f.read()
with open('../gsm2py/data/ta_directive','r') as f:
    ta_dir=f.read()
# through the openai API i do not have access to o1-mini it seems :( 
llm='gpt-4o'
m=model.Model(llm,enc_dir,dec_dir,ta_dir)

first run a couple of responses using the GSMResponse format to structure the output
=
`GSMResponse` has the following format
```
class GSMResponse(BaseModel):
    gsm: str [input GSM]
    reasoning: str [reasoning
    pycode: str = [python def of the generated solution sol]
    answer: float = [the final answer to the math problem calculated by calling the function sol (in principle)]
    IC: str = [Irrelevant Context if any]
    testcase: str = [test to see the mapping between sol variables and problem inputs]
```

in addition `GSMResponse` has a `validate` method (returning bool) which explicitly calls the test case and compares the answer vs the answer field to check consistency

In [21]:
gsm1='When Sophie watches her nephew, she gets out a variety of toys for him. The bag of building blocks has 31 blocks in it. The bin of stuffed animals has 8 stuffed animals inside. The tower of stacking rings has 9 multicolored rings on it.Sophie recently bought a tube of bouncy balls, bringing her total number of toys for her nephew up to 62. How many bouncy balls came in the tube?'
gsm2='To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?'
gsm2_1='To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. I eat my lunch in 15 minutes every day.After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?'

# this problem appears in gsm-symbolic paper and o1-mini fails on it
# you can check it fails due to the no-op if you choose the model gpt-4o-mini with the same mistake
gsm3='Oliver picks 44 kiwis on Friday. Then he picks 58 kiwis on Saturday. On Sunday, he picks double the number of kiwis he did on Friday, but five of them were a bit smaller than average. How many kiwis does Oliver have?'

no IC
--

In [22]:
ans1=m.dec(gsm2, 
          #followup='does your function sol satisfy directive 0.5 ?',
          #followup2='why the directives were not sufficient to have you fully optimize the function sol ?',
          #followup2='have you followed directive 7.4 ?'
         )
print(ans1['A1'].validate())
print(ans1['A1'])

True
{
    "gsm": "To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?",
    "reasoning": "To solve this problem, first calculate the total cost without discount, based on the tiered pricing structure:
- For the first 10 minutes: 10 * $0.6 = $6
- For minutes 11-25, which are 15 minutes: 15 * $0.5 = $7.5
- For minutes starting at 26 to 60, which are 35 minutes: 35 * $0.3 = $10.5

Sum all these amounts to get the total cost: $6 + $7.5 + $10.5 = $24.

Since the bill is more than $10, apply a 25% discount to the total: 25% of $24 is $6, reducing the cost to $24 - $6 = $18.",
    "clues": [
        "Cost per minute varies with time periods (0-10 min, 11-25 min, 26+ min).",
        "Total bill gets a 25% discount if 

asking help from the TA
---

In [24]:
print(ans1b['A1'])

{
    "gsm": "To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?",
    "reasoning": "We calculate the cost of a 60-minute phone call according to the rules given in the problem. First, we separate the duration into the specified segments: the first 10 minutes, the next 15 minutes, and the final 35 minutes. Each segment has its price per minute. We apply these rates to the durations to find the total cost before discount. If the total cost is more than $10, a discount is applicable, so we recalculate the final cost as 75% of the total cost.",
    "clues": [
        "Cost per minute for the first 10 minutes is $0.6.",
        "Cost per minute for minutes 11 to 25 is $0.5.",
        "Cost per minute after 25 minu

In [23]:
ans1b=m.dec(gsm2, ta=True)
print(ans1b['A1b'].validate())
print(ans1b['A1b'])

False
{
    "gsm": "To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?",
    "reasoning": "We calculate the cost of a 60-minute phone call according to the rules given in the problem. First, we separate the duration into the specified segments: the first 10 minutes, the next 15 minutes, and the final 35 minutes. Each segment has its price per minute. We apply these rates to the durations to find the total cost before discount. If the total cost is more than $10, a discount is applicable, so we recalculate the final cost as 75% of the total cost. For the first 10 minutes, the cost is 10 * $0.6 = $6, for the next 15 minutes, the cost is 15 * $0.5 = $7.5, for the remaining 35 minutes, the cost is 35 * $0.3 = $10.

In [15]:
ans1b['A1'].validate()

False

with IC
--

In [4]:
ans2=m.dec(gsm2_1, 
          #followup='does your function sol satisfy directive 0.5 ?',
          followup='challenge whether the noise was detected correctly',
          #followup2='why the directives were not sufficient to have you fully optimize the function sol ?',
          #followup2='have you followed directive 7.4 ?'
         )
print(ans2['A1'].validate())
print(ans2['A1'])

False
{
    "gsm": "To make a call from a hotel room phone, you must pay $0.6 for each minute of your call. After 10 minutes, the price drops to $0.5 per minute. After 25 minutes from the start of the call, the price drops even more to $0.3 per minute. If your total bill is more than $10, you get a 25% discount. How much would a 60-minute call cost?",
    "reasoning": "First, we calculate the cost for each segment of the call based on the rate changes. For the first 10 minutes, the cost is at $0.6 per minute. From 10 to 25 minutes, the cost is reduced to $0.5 per minute. Finally, from 25 to 60 minutes, the cost is at $0.3 per minute. We sum these costs to find the total cost of the call. If this cost exceeds $10, we apply a 25% discount, reducing the total by that percentage.",
    "clues": [
        "The rate for the first 10 minutes is $0.6 per minute.",
        "The rate from 11 to 25 minutes is $0.5 per minute.",
        "The rate from 26 to 60 minutes is $0.3 per minute.",
       

In [5]:
print(ans2['A2'])

{
    "generic": "The sentence "I eat my lunch in 15 minutes every day." was marked as noise correctly because it is not relevant to the calculation of the cost of a phone call, which is the core problem to be solved. The sentence describes a personal habit and does not relate to phone call durations or rates, which are the only factors that impact the solution.",
    "reasoning": "The identified noise ("I eat my lunch in 15 minutes every day.") is unrelated to any variable or factor involved in computing the cost of the phone call. Hence, excluding this information is justified as it does not contribute to the problem's calculation logic or solution.",
    "pytests": "
def test_sol():
    # Testing with initial conditions
    assert sol(call_duration=60, rate1_duration=10, rate1_rate=0.6, rate2_duration=15, rate2_rate=0.5, rate3_rate=0.3, discount_threshold=10, discount_rate=0.25) == 13.5
    # Testing with an exact boundary condition for rate change and discount just reaching thresho

misc: check some directives are followed
--

In [15]:
ans3=m.dec(gsm2, 
          followup='does your function sol satisfy directive 0.5 ?',
          #followup2='why the directives were not sufficient to have you fully optimize the function sol ?',
          followup2='have you followed directive 7.4 ?'
         )
print(ans3['A1'].validate())
print(ans3['A3'])

True
{
    "generic": "Yes, directive 7.4 has been followed as the function `sol` is refactored to avoid hard-coded values. Instead, it uses parameters to accommodate variables like rate changes, discount rates, and minute thresholds. This adjustment conforms to directive 7.4’s requirement to ensure the function is not dependent on fixed constants, thereby increasing its versatility and applicability to different scenarios that might use different pricing structures or thresholds.",
    "reasoning": "By refactoring the function to make constants such as the rates per minute, thresholds for rate changes, and the discount conditions as parameter inputs, `sol` complies with directive 7.4. This change promotes flexibility and ensures it can adapt to various similar problems with different numerical values without the need for additional changes in the function's internal logic. Thus, it meets the goal of the directive to make the function as parameter-driven and adaptable as possible.",
  

let's generate more problems based on this function
===

In [66]:
def mod(total_minutes, rate1, break1, rate2, break2, rate3, discount_threshold, discount_rate):
    if total_minutes <= break1:
        cost = total_minutes * rate1
    elif total_minutes <= break2:
        cost = (break1 * rate1) + ((total_minutes - break1) * rate2)
    else:
        cost = (break1 * rate1) + ((break2 - break1) * rate2) + ((total_minutes - break2) * rate3)
    
    if cost > discount_threshold:
        cost *= (1 - discount_rate)
    
    return cost

# Constants for the specific problem
total_minutes = 60
rate1 = 0.6
break1 = 10
rate2 = 0.5
break2 = 25
rate3 = 0.3
discount_threshold = 10
discount_rate = 0.25

# Calculate and print the cost of a 60-minute call
call_cost =mod(total_minutes, rate1, break1, rate2, break2, rate3, discount_threshold, discount_rate)
call_cost

18.0

In [59]:
sol(total_minutes, discount_threshold, rate1, break1, rate2, break2-break1, rate3, discount_rate)

NameError: name 'sol' is not defined

In [67]:
gsm=m.enc(mod)

In [68]:
print(gsm)

{
    "pycode": "
def mod(total_minutes, rate1, break1, rate2, break2, rate3, discount_threshold, discount_rate):
    if total_minutes <= break1:
        cost = total_minutes * rate1
    elif total_minutes <= break2:
        cost = (break1 * rate1) + ((total_minutes - break1) * rate2)
    else:
        cost = (break1 * rate1) + ((break2 - break1) * rate2) + ((total_minutes - break2) * rate3)
    
    if cost > discount_threshold:
        cost *= (1 - discount_rate)
    
    return cost",
    "question": "You are planning to rent a bike from a rental shop. The shop charges at different rates based on how long you rent the bike. 

- For the first 30 minutes, the rate is $0.50 per minute.
- From minute 31 to minute 60, the rate is $0.40 per minute.
- For any additional minutes after 60, the rate is $0.30 per minute.

The rental shop also offers a discount: if the total cost exceeds $25.00, they apply a 10% discount on the total cost. You decide to rent a bike for 90 minutes. How much will

In [24]:
print(gsm.content)

<GSM>Problem 1: Sarah runs a tutoring service where she charges according to the number of minutes of tutoring provided. For the first 30 minutes, she charges $0.50 per minute. After the first 30 minutes and up to 60 minutes, she charges $0.70 per minute. For any time beyond 60 minutes, she charges $0.90 per minute. If the total charged before any discount exceeds $40, she offers a discount of 10%. One day, Sarah provided tutoring for a total of 75 minutes. Calculate the total cost for this session after applying any applicable discount. Express your answer in dollars.</GSM>

<ANALYSIS>
6.1.1) The problem context specifies different rates for different intervals of time, corresponding to the conditions in the function.
6.1.2) Addition is used to accumulate total cost from different time intervals, which makes sense as each interval adds to the total charge.
6.1.3) No subtraction is used in this problem, aligning well with the context which does not involve any deductions other than the

ok let's put one of the answers into our decoder
===

In [41]:
gsm2b='''Greg owns a kiosk at a mall where he offers game console sessions for people to play. 
He charges $0.20 per minute for the first 15 minutes, $0.35 per minute for the next 30 minutes, 
and $0.50 per minute for any additional time. On a busy Saturday, Greg had a customer who played for 55 minutes. 
If the total cost of a session exceeds $15, Greg provides a 5% discount. 
Additionally, during the same day, Greg noted that 20 people visited his kiosk and there was a sale on headphones 
which had a 20% discount, but these details are unrelated to the task. 
Calculate the total cost for the 55-minute session after any applicable discount, in dollars.'''




In [91]:
#gsm2b="Jessica is planning to plant some new flowers in her garden. She's browsing through a catalog that lists prices for multiple bulk purchases. The first set of up to 10 seeds costs $0.60 per seed. If she buys between 11 to 25 seeds, the cost drops to $0.50 per seed for each additional seed beyond 10. For any seeds over 25, the cost goes down further to $0.30 per seed. If the total cost of her seed purchase surpasses $10, she gets a 25% discount on the total cost. Meanwhile, as she was looking through the catalog, she noticed there were also discounts on gardening tools and fertilizers, but those do not apply to her seed purchase. Calculate her total cost (in dollars) for buying 28 seeds."
# gsm2b='''A new small bakery chain calculates special delivery service fees based on delivery time. The rates are structured as follows:

# - The first 60 minutes cost $10 per minute.
# - The following 30 minutes cost $12 per minute.
# - Any time thereafter costs $15 per minute.

# If a delivery exceeds the cost threshold of $900, a 15% discount is offered on the total fee.

# On a particularly eventful day, the bakery made deliveries spanning 120 minutes. Calculate the bakery's total fee for this service.

# (Note: The bakery also received an order that was canceled after preparation, which would have taken an additional 45 minutes).

# What is the total delivery fee (in dollars) in this case?'''

In [19]:
# gsm2b='''
# Maria is a long-distance runner who participates in running sessions with varying speeds based on her location. For her morning session, she has a path divided into three segments. In the first segment, she runs at a speed of 4 meters per minute for the first 20 minutes. Once she passes the 20-minute mark, her speed changes to 5 meters per minute until she reaches the 40-minute mark, which marks the end of the second segment. Finally, in the third segment beyond 40 minutes, she speeds up to 6 meters per minute.

# Maria plans to run for a total of 45 minutes today. However, she has a joint discount offer with the gym where if her calculated total distance exceeds 220 meters, she gets a 10% reduction in her monthly membership fee. Help Maria calculate her potentially discounted monthly membership fee assuming her regular monthly fee is $400.

# 1. Determine the distance Maria covers in each segment.
# 2. Calculate the total distance.
# 3. Apply the discount rule to find out her final monthly fee.

# What will Maria's final monthly fee be?
# '''

In [61]:
res=m.dec(gsm2b,mod=mod)

here is the answer with noise nicely left out
===

In [62]:
print(res['A1'])

To solve the problem, we will calculate the cost of the 55-minute session based on the given pricing structure and apply any discounts if necessary. Here are the steps broken down:

1. **Pricing Structure**:
   - First 15 minutes: $0.20 per minute.
   - Next 30 minutes: $0.35 per minute.
   - Additional time beyond the first 45 minutes: $0.50 per minute.

2. **Calculate Session Cost**:
   - Calculate cost for the first 15 minutes.
   - Calculate cost for the next 30 minutes.
   - Calculate cost for the remaining 10 minutes (since the total session is 55 minutes).

3. **Discount**:
   - Check if the total cost exceeds $15.
   - If it does, apply a 5% discount.

4. **Extraneous Information**:
   - The information about the number of people visiting the kiosk, the sale on headphones, and any non-calculation related context are irrelevant to the solution of this problem.

Now, let's put this into a function:

```python
def sol(total_minutes, first_rate, first_duration, second_rate, second_

In [63]:
print(res['A2'])

To determine if `sol` is functionally equivalent to `mod`, we must first identify the mapping between the function arguments and ensure that the calculations are analogous given this mapping.

Here's the mapping for the arguments:
- \( \text{rate1} \equiv \text{first_rate} \)
- \( \text{break1} \equiv \text{first_duration} \)
- \( \text{rate2} \equiv \text{second_rate} \)
- \( \text{break2} \equiv \text{first_duration} + \text{second_duration} \)
- \( \text{rate3} \equiv \text{additional_rate} \)

Both functions take `total_minutes`, `discount_threshold`, and `discount_rate` as equivalent.

### Unit Tests

We will write three-unit tests to confirm that both functions yield the same results for a range of scenarios.

```python
def test_equivalence():
    # Test 1: Within the first rate
    total_minutes = 10
    first_rate = 0.20
    first_duration = 15
    second_rate = 0.35
    second_duration = 30
    additional_rate = 0.50
    discount_threshold = 15
    discount_rate = 0.05

    re

In [70]:
def sol(total_minutes, first_rate, first_duration, second_rate, second_duration, additional_rate, discount_threshold, discount_rate):
    # Cost calculations
    if total_minutes <= first_duration:
        cost = total_minutes * first_rate
    elif total_minutes <= first_duration + second_duration:
        cost = (first_duration * first_rate) + (total_minutes - first_duration) * second_rate
    else:
        cost = (first_duration * first_rate) + \
               (second_duration * second_rate) + \
               (total_minutes - first_duration - second_duration) * additional_rate

    # Apply discount if applicable
    if cost > discount_threshold:
        cost -= cost * discount_rate / 100

    return cost
    
def test_equivalence():
    # Test 1: Within the first rate
    total_minutes = 10
    first_rate = 0.20
    first_duration = 15
    second_rate = 0.35
    second_duration = 30
    additional_rate = 0.50
    discount_threshold = 15
    discount_rate = 0.05

    result_sol = sol(
        total_minutes=total_minutes,
        first_rate=first_rate,
        first_duration=first_duration,
        second_rate=second_rate,
        second_duration=second_duration,
        additional_rate=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=100 * discount_rate
    )

    result_mod = mod(
        total_minutes=total_minutes,
        rate1=first_rate,
        break1=first_duration,
        rate2=second_rate,
        break2=first_duration + second_duration,
        rate3=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=discount_rate
    )

    assert result_sol == result_mod, f"Test 1 failed: {result_sol} != {result_mod}"

    # Test 2: Spanning first and second rates
    total_minutes = 40

    result_sol = sol(
        total_minutes=total_minutes,
        first_rate=first_rate,
        first_duration=first_duration,
        second_rate=second_rate,
        second_duration=second_duration,
        additional_rate=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=100 * discount_rate
    )

    result_mod = mod(
        total_minutes=total_minutes,
        rate1=first_rate,
        break1=first_duration,
        rate2=second_rate,
        break2=first_duration + second_duration,
        rate3=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=discount_rate
    )

    assert result_sol == result_mod, f"Test 2 failed: {result_sol} != {result_mod}"

    # Test 3: Using all rates and applying discount
    total_minutes = 55

    result_sol = sol(
        total_minutes=total_minutes,
        first_rate=first_rate,
        first_duration=first_duration,
        second_rate=second_rate,
        second_duration=second_duration,
        additional_rate=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=100 * discount_rate
    )

    result_mod = mod(
        total_minutes=total_minutes,
        rate1=first_rate,
        break1=first_duration,
        rate2=second_rate,
        break2=first_duration + second_duration,
        rate3=additional_rate,
        discount_threshold=discount_threshold,
        discount_rate=discount_rate
    )

    assert result_sol == result_mod, f"Test 3 failed: {result_sol} != {result_mod}"

test_equivalence()